# Парсинг с помощью BeautifulSoup

In [ ]:
 1.99: 0.803

Beautiful Soup - это библиотека для Python, которая позволяет парсить (анализировать) HTML и XML документы. Она предоставляет удобный способ искать, навигировать, и модифицировать дерево DOM (Document Object Model), представляющее HTML/XML документ.

# Задание

Вам необходимо собрать датасет, спарсив данные из этого сайта:

https://books.toscrape.com/

Всего на сайте 1000 книг. То есть длина датасета должна равняться количеству книг.
 
Итоговая таблица должна содержать следующие столбцы:

| Название столбца | Описание | 
|--|--|
|id| Идентификатор книги |
|book_name| Название книги |
|price| Цена в £ |
|stock| Наличие книги. 1 или 0|
|url| Ссылка на книгу |

**Примечание по столбцам:**
- `id` - заполняется разработчиком датасета. Первая спарсенная книга имеет `id` = `0`.
- `url` - должна содержать полную ссылку. Не только конец ссылки, указанный на сайте. То есть по данному url можно перейти одним кликом.

## Импорт библиотек

In [1]:
# ваш код здесь
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

## Cоздание датасета и парсинг данных

In [2]:
# ваш код здесь
url = 'https://books.toscrape.com/'

In [3]:
df = pd.DataFrame({
    # 1
    'id':[],
    'book_name':[],
    'price':[],
    'stock':[],
    'url':[],
    # 2
    'num_of_rev':[],
    'desc':[]
})

In [4]:
r = requests.get(url)
r

<Response [200]>

In [5]:
soup = BeautifulSoup(r.content, 'html.parser')

In [6]:
soup

<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link href="s

In [7]:
def get_books_set(url=url, data=df):
    lenta = {}
    for n in range(1, 51):
        url_to_scrape = url.format(n)
        result = requests.get(url_to_scrape)
        soup = BeautifulSoup(result.text, 'lxml')
        pages = soup.select('.product_pod')
    
        for page in pages:
            # 1
            lenta['book_name'] = page.select('a')[1]['title']
            lenta['price'] = page.find('p', class_='price_color').text[1:]
            
            if 'In stock' in str(page.find('p', class_="instock availability")):
                lenta['stock'] = 1
            else:
                lenta['stock'] = 0

            lenta['url'] = url_page = url + page.select('a')[1]['href']
            
            # 2
            r = requests.get(url_page)
            soup_page = BeautifulSoup(r.text, 'lxml')
            tags = soup_page.find_all('p')
            for tag in tags:
                text = " ".join(tag.text.split())
                if 'In stock (' in text:
                    match = re.search(r'\d+', text)
                    if match:
                        lenta['num_of_rev'] = int(match.group())
                elif len(text) > 30:
                    lenta['desc'] = text
            
            df.loc[len(df)] = lenta

    df['id'] = df.index

In [8]:
get_books_set(url, df)

## Итоговый датасет

In [9]:
df.shape

(1000, 7)

In [10]:
display(
    df.head(),
    df.tail()
)

,id,book_name,price,stock,url,num_of_rev,desc
0,0,A Light in the Attic,£51.77,1,https://books.toscrape.com/catalogue/a-light-i...,22,It's hard to imagine a world without A Light i...
1,1,Tipping the Velvet,£53.74,1,https://books.toscrape.com/catalogue/tipping-t...,20,"""Erotic and absorbing...Written with starling ..."
2,2,Soumission,£50.10,1,https://books.toscrape.com/catalogue/soumissio...,20,"Dans une France assez proche de la nÃ´tre, un ..."
3,3,Sharp Objects,£47.82,1,https://books.toscrape.com/catalogue/sharp-obj...,20,"WICKED above her hipbone, GIRL across her hear..."
4,4,Sapiens: A Brief History of Humankind,£54.23,1,https://books.toscrape.com/catalogue/sapiens-a...,20,From a renowned historian comes a groundbreaki...


,id,book_name,price,stock,url,num_of_rev,desc
995,995,Our Band Could Be Your Life: Scenes from the A...,£57.25,1,https://books.toscrape.com/catalogue/our-band-...,19,This is the never-before-told story of the mus...
996,996,Olio,£23.88,1,https://books.toscrape.com/catalogue/olio_984/...,19,"Part fact, part fiction, Tyehimba Jess's much ..."
997,997,Mesaerion: The Best Science Fiction Stories 18...,£37.59,1,https://books.toscrape.com/catalogue/mesaerion...,19,"Andrew Barger, award-winning author and engine..."
998,998,Libertarianism for Beginners,£51.33,1,https://books.toscrape.com/catalogue/libertari...,19,Libertarianism isn't about winning elections; ...
999,999,It's Only the Himalayas,£45.17,1,https://books.toscrape.com/catalogue/its-only-...,19,"âWherever you go, whatever you do, just . . ..."


# ЗАДАНИЕ ПРО

Так, мы спарсили данные о книгах. Но данные какие-то неполные. Часть названия стирается из-за отображения и нет ни полного названия книги, ни описания этой книги, ни жанра.

Вам необходимо дополнить датасет, спарсив дополнительные данные из того же сайта:

https://books.toscrape.com/
 
Итоговая таблица должна содержать следующие столбцы:

| Название столбца | Описание | 
|--|--|
|id| Идентификатор книги |
|book_name| Название книги - только полное название|
|genre| жанр книги |
|desc| описание |
|price| Цена в £ |
|stock| Наличие книги. 1 или 0|
|url| Ссылка на книгу |
| num_of_rev | количество отзывов|

## Парсинг данных и обогащение датасета

## Итоговый датасет PRO

In [11]:
df.shape

(1000, 7)

In [12]:
display(
    df.head(),
    df.tail()
)

,id,book_name,price,stock,url,num_of_rev,desc
0,0,A Light in the Attic,£51.77,1,https://books.toscrape.com/catalogue/a-light-i...,22,It's hard to imagine a world without A Light i...
1,1,Tipping the Velvet,£53.74,1,https://books.toscrape.com/catalogue/tipping-t...,20,"""Erotic and absorbing...Written with starling ..."
2,2,Soumission,£50.10,1,https://books.toscrape.com/catalogue/soumissio...,20,"Dans une France assez proche de la nÃ´tre, un ..."
3,3,Sharp Objects,£47.82,1,https://books.toscrape.com/catalogue/sharp-obj...,20,"WICKED above her hipbone, GIRL across her hear..."
4,4,Sapiens: A Brief History of Humankind,£54.23,1,https://books.toscrape.com/catalogue/sapiens-a...,20,From a renowned historian comes a groundbreaki...


,id,book_name,price,stock,url,num_of_rev,desc
995,995,Our Band Could Be Your Life: Scenes from the A...,£57.25,1,https://books.toscrape.com/catalogue/our-band-...,19,This is the never-before-told story of the mus...
996,996,Olio,£23.88,1,https://books.toscrape.com/catalogue/olio_984/...,19,"Part fact, part fiction, Tyehimba Jess's much ..."
997,997,Mesaerion: The Best Science Fiction Stories 18...,£37.59,1,https://books.toscrape.com/catalogue/mesaerion...,19,"Andrew Barger, award-winning author and engine..."
998,998,Libertarianism for Beginners,£51.33,1,https://books.toscrape.com/catalogue/libertari...,19,Libertarianism isn't about winning elections; ...
999,999,It's Only the Himalayas,£45.17,1,https://books.toscrape.com/catalogue/its-only-...,19,"âWherever you go, whatever you do, just . . ..."
